# Japan Search Gallery client

> Tools to interact with the Japan Search Gallery

In [ ]:
#| default_exp api

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from pprint import pprint
import os
import json

class Component:
    
    def export(self):
        '''
        エクスポート

        Returns:  
        * dict containing representation of the gallery
        '''
        return self.item

    def setMetadata(self, key, lang, value):
        '''
        メタデータの設定

        Parameters:  
        * `key` - title, description, etc...  
        * `lang` - one of 言語, eg: 'ja', 'en', 'yomi'  
        * `value` - 値
        '''
        item = self.item
        if key not in item:
            item[key] = {}
        item[key][lang] = value


    def save(self, path):
        '''
        JSONファイルの保存

        Parameters:  
        * `path` - 出力先のパス
        '''
        os.makedirs(os.path.dirname(path), exist_ok=True)
        with open(path, 'w') as f:
            json.dump(self.item, f, ensure_ascii=False, indent=2)

class Media(Component):

    def __init__(self, type, title, source):
        self.item = {
            "imgType": type,
            "cover": None,
            "sourceUrl": None,
            "title": {
                "ja": title
            },
            "source": {
                "ja": source
            },
            "contentsRightsType": "various",
        }

class IIIF(Media):
    def __init__(self, manifestUrl, infoJsonUrl, title, source):
        super().__init__("IIIF", title, source)

        item = self.item
        item["cover"] = None
        item["sourceUrl"] = None
        item["manifestUrl"] = manifestUrl
        item["infoJsonUrl"] = infoJsonUrl

class UrlImage(Media):
    """
    URLを指定して表示する画像に関するクラス
    """

    def __init__(self, url, title, source, thumbnailUrl=None):
        """
        初期化

        Parameters:  
        * `url` - 画像URL  
        * `title` - 日本語のタイトル  
        * `source` - 日本語の出典  
        * `thumbnailUrl` - サムネイルURL
        """
        super().__init__("URL", title, source)

        item = self.item

        item["url"] = url
        if thumbnailUrl is not None:
            item["thumbnailUrl"] = thumbnailUrl



class Part(Component):

    def __init__(self, type):
        item = {}
        self.item = item
        item["cpid"] = "a"
        item["type"] = type

    def insertText(self, lang, value):
        '''
        削除予定
        '''
        item = self.item

        if lang not in item:
            item[lang] = {
                "ops": []
            }

        ops = item[lang]["ops"]
        ops.append({
            "insert": value
        })
        ops.append({
            "insert": "\n"
        })

    def addOps(self, key, lang, value):
        '''
        opsに要素を追加します。

        Parameters:  
        * `key` - title, description, etc...  
        * `lang` - one of 言語, eg: 'ja', 'en', 'yomi'  
        * `value` - 値
        '''
        item = self.item
        if key not in item:
            item[key] = {}

        if lang not in item[key]:
            item[key][lang] = {
                "ops": []
            }

        ops = item[key][lang]["ops"]
        ops.append({
            "insert": value
        })
        ops.append({
            "insert": "\n"
        })

    def setMetadata(self, key, lang, value):
        item = self.item
        if key not in item:
            item[key] = {}

        # self.item[key] = value
        item[key][lang] = value

    def setUrl(self, url):
        self.item["url"] = url

    def addPart(self, part):
        self.item["parts"].append(part.export())    

    def setImage(self, image):
        self.item["image"] = image.export()

class ListPart(Part):
    '''
    リストパーツ
    '''
    def __init__(self, viewType):
        '''
        初期化

        Parameters:  
        * `viewType` - レイアウト, eg: 'tile', 'table', etc...
        '''
        super().__init__("list") # , viewType

        item = self.item

        item["viewType"] = viewType

        item["parts"] = []

class ListExternalLinkPart(Part):
    '''
    ListExternalLinkPart
    '''
    
    def __init__(self):
        '''
        初期化
        '''
        super().__init__("jps-curation-list-external-link")

    
    def setUrl(self, url):
        '''
        外部リンクのURLを設定します。

        Parameters:  
        * `url` - 外部リンクのURL
        '''
        self.item["url"] = url

class JPSGalleryClient(Component):

    item = {
        "version": 1,
        "seqNo": 0,
        "primaryTerm": 1,
        "curationType": "GALLERY",
        "title": {},
        "summary": {},
        "responsibility": {},
        "workspace": "abc",
        "workspaceTags": [],
        "permission": {
            "publicationLevel": "WORKSPACE",
            "level": "ADMIN"
        },
        "edit": {
            "userId": "system",
            "orgId": "jpsws",
            "date": 1663722096491,
            "action": "CREATE"
        },
        "parts": []
    }

    def __init__(self):
        self.item["id"] = "aaa"

    def setTitle(self, key, value):
        self.item["title"][key] = value

    def setSummary(self, key, value):
        self.item["summary"][key] = value

    

    def addTag(self, tag):
        '''
        タグを追加します。

        Parameters:  
        * `tag` - タグ
        '''
        item = self.item
        item["workspaceTags"].append(tag)

    def setImage(self, image):
        self.item["image"] = image.export()

    def saveJsonl(self, path):
        '''
        JSONLファイルの保存

        Parameters:  
        * `path` - 出力先のパス
        '''

        arr = []
        arr.append(json.dumps(self.item, ensure_ascii=False))

        os.makedirs(os.path.dirname(path), exist_ok=True)
        with open(path, mode='w') as f:
            f.write("\n".join(arr))

First import the JPSGalleryClient class.
```
from jps_gallery_tools.api import JPSGalleryClient
```
To initialise the JPSGalleryClient you need to provide the base url of your Omeka instance's API.

In [ ]:
gallery = JPSGalleryClient()

## ギャラリーのメタデータの設定

ギャラリーのメタデータを設定します。

In [ ]:
show_doc(JPSGalleryClient.setMetadata)

---

### Component.setMetadata

>      Component.setMetadata (key, lang, value)

メタデータの設定

Parameters:  
* `key` - title, description, etc...  
* `lang` - one of 言語, eg: 'ja', 'en', 'yomi'  
* `value` - 値

タイトルの設定

In [ ]:
gallery.setMetadata("title", "ja", "日本語タイトル")
gallery.setMetadata("title", "en", "English Title")
gallery.setMetadata("title", "yomi", "にほんごのよみ")

summaryの設定

In [ ]:
gallery.setMetadata("summary", "ja", "日本語の概要文")
gallery.setMetadata("summary", "en", "English Summary")

responsibilityの設定

In [ ]:
gallery.setMetadata("responsibility", "ja", "中村覚")
gallery.setMetadata("responsibility", "en", "Satoru Nakamura")

tagの設定

In [ ]:
show_doc(JPSGalleryClient.addTag)

---

### JPSGalleryClient.addTag

>      JPSGalleryClient.addTag (tag)

タグを追加します。

Parameters:  
* `tag` - タグ

In [ ]:
gallery.addTag("はじめてのタグ")
gallery.addTag("二つ目のタグ")

画像の設定

In [ ]:
show_doc(UrlImage)

---

### UrlImage

>      UrlImage (url, title, source, thumbnailUrl=None)

URLを指定して表示する画像に関するクラス

In [ ]:
show_doc(UrlImage.__init__)

---

### UrlImage.__init__

>      UrlImage.__init__ (url, title, source, thumbnailUrl=None)

初期化

Parameters:  
* `url` - 画像URL  
* `title` - 日本語のタイトル  
* `source` - 日本語の出典  
* `thumbnailUrl` - サムネイルURL

In [ ]:
original_url = "https://blogger.googleusercontent.com/img/a/AVvXsEiTjjaWkCHeCslxeKujaQotjuR_HvhOHen7Ql7rQV8nmQl4ivLRF_CSabMY-JLJHNL-xHMe3VDyA9F-iv1RtQ2jFTk1Vnjck-veKLUZPdYqCcjzHkc8SGKb1fyZ_x8W66aTrSXQ78UN3CnyBakwwmmH_hmPKeEsTzq8J3e2zzUMylmDjCZ0LClPByBRRw=s400"
thumbnail_url = "https://blogger.googleusercontent.com/img/a/AVvXsEiTjjaWkCHeCslxeKujaQotjuR_HvhOHen7Ql7rQV8nmQl4ivLRF_CSabMY-JLJHNL-xHMe3VDyA9F-iv1RtQ2jFTk1Vnjck-veKLUZPdYqCcjzHkc8SGKb1fyZ_x8W66aTrSXQ78UN3CnyBakwwmmH_hmPKeEsTzq8J3e2zzUMylmDjCZ0LClPByBRRw=s180-c"
title = "初日の出のキャラクター"
source = "いらすとや"
top_image = UrlImage(original_url, title, source, thumbnail_url)

In [ ]:
show_doc(JPSGalleryClient.setImage)

---

### JPSGalleryClient.setImage

>      JPSGalleryClient.setImage (image)

In [ ]:
gallery.setImage(top_image)

## パーツの追加

In [ ]:
show_doc(ListPart.__init__)

---

### ListPart.__init__

>      ListPart.__init__ (viewType)

初期化

Parameters:  
* `viewType` - レイアウト, eg: 'tile', 'table', etc...

In [ ]:
part = ListPart("tile")

メタデータの追加

In [ ]:
part.setMetadata("title", "ja", "リストのタイトル")
part.setMetadata("title", "en", "List Title")

リストに画像の追加

In [ ]:
item = ListExternalLinkPart()

In [ ]:
show_doc(ListExternalLinkPart.setUrl)

---

### ListExternalLinkPart.setUrl

>      ListExternalLinkPart.setUrl (url)

外部リンクのURLを設定します。

Parameters:  
* `url` - 外部リンクのURL

In [ ]:
item.setUrl("https://www.irasutoya.com/2021/11/blog-post_91.html")

アイテムの追加

In [ ]:
show_doc(ListPart.addPart)

---

### Part.addPart

>      Part.addPart (part)

In [ ]:
part.addPart(item)

In [ ]:
pprint(part.item)

{'cpid': 'a',
 'parts': [{'cpid': 'a',
            'type': 'jps-curation-list-external-link',
            'url': 'https://www.irasutoya.com/2021/11/blog-post_91.html'}],
 'title': {'en': 'List Title', 'ja': 'リストのタイトル'},
 'type': 'list',
 'viewType': 'tile'}


## Save json

設定した内容をjsonファイルに出力します。

In [ ]:
show_doc(JPSGalleryClient.save)

---

### Component.save

>      Component.save (path)

JSONファイルの保存

Parameters:  
* `path` - 出力先のパス

In [ ]:
gallery.save("tmp/gallery.json")

In [ ]:
show_doc(JPSGalleryClient.saveJsonl)

---

### JPSGalleryClient.saveJsonl

>      JPSGalleryClient.saveJsonl (path)

JSONLファイルの保存

Parameters:  
* `path` - 出力先のパス

In [ ]:
gallery.saveJsonl("tmp/gallery.jsonl")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()